In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from pprint import pprint
import pandas as pd

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.bn_db
collection = db.metadata

In [127]:
url = 'http://search.barnesandnoble.com/bookSearch/isbnInquiry.asp?isbn=9780761352556#/'

response = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.120 Safari/537.36'})


soup = BeautifulSoup(response.content,'lxml')

# results are returned as an iterable list

current_price = soup.find_all('span', class_='price current-price ml-0')[0].text
list_price = soup.find_all('s', class_='old-price')[0].text

rating_value = soup.find_all("span", itemprop="ratingValue")[0].text
review_count = soup.find_all("span", itemprop="reviewCount")[0].text
high_value = soup.find_all("span", itemprop="bestRating")[0].text
low_value = soup.find_all("span", itemprop="worstRating")[0].text
table_rows = soup.find_all('table', class_="plain centered")[0].text

In [128]:
df = pd.read_html(str(table))[0]
df1 = pd.DataFrame(df)
df1 = df1.set_index(0)
df_dict = df1.to_dict()

In [133]:
    # Dictionary to be inserted into MongoDB
    mongo_dict = {
        'Sale_Price': current_price,
        'List_price': list_price,
        'Average_Rating' : rating_value,
        'Review_Count' : review_count,
        'Highest_Rating' : high_value,
        'Lowest_Rating' : low_value,
        'Product_details': df_dict,
    }

pprint(mongo_dict)

{'Average_Rating': '4.2',
 'Highest_Rating': '5',
 'List_price': '$17.99',
 'Lowest_Rating': '1',
 'Product_details': {1: {'Age Range:': '8 - 12 Years',
                         'Edition description:': 'Library Edition',
                         'ISBN-13:': '9780761352556',
                         'Lexile:': "700L (what's this?)",
                         'Pages:': '32',
                         'Product dimensions:': '9.20(w) x 11.10(h) x 0.40(d)',
                         'Publication date:': '08/01/2010',
                         'Publisher:': 'Lerner Publishing Group',
                         'Sales rank:': '46492',
                         'Series:': 'Carolrhoda Picture Bks'}},
 'Review_Count': '5',
 'Sale_Price': '$17.87'}


In [130]:
# Display the MongoDB records created above
details = db.metadata.find()
for detail in details:
    print(detail)